Model Development and Evaluation

Notebook Purpose
Build, train, and compare multiple machine learning models to predict Titanic survival.

Input
- `train_features.csv` - Training data with engineered features
- `test_features.csv` - Test data with engineered features
- `test_passenger_ids.csv` - Passenger IDs for submission
 Output
- Trained model files (`.pkl`)
- Model comparison metrics
- Feature importance visualizations
- Kaggle submission file

Models to Evaluate
1. **Logistic Regression** - Baseline linear model
2. **Random Forest** - Ensemble of decision trees
3. **XGBoost** - Gradient boosting (often wins competitions)

In [1]:
# Initial Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn imports
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)

# XGBoost
from xgboost import XGBClassifier

# Model persistence
import joblib

In [2]:
# Set up visualization options
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

In [3]:
# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

Load Feature-Engineered Data

Loading from the previous checkpoint - our feature engineering outputs.

In [4]:
# Load the feature-engineered datasets
train_df = pd.read_csv('../data/processed/train_features.csv')
test_df = pd.read_csv('../data/processed/test_features.csv')
test_ids = pd.read_csv('../data/processed/test_passenger_ids.csv')

print(f"Training set: {train_df.shape}")
print(f"Test set: {test_df.shape}")
print(f"\nTraining columns: {train_df.columns.tolist()}")

Training set: (891, 35)
Test set: (418, 34)

Training columns: ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_encoded', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Deck_A', 'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'Deck_T', 'Deck_Unknown', 'FamilySize', 'IsAlone', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare', 'Age_Child', 'Age_Teenager', 'Age_Young_Adult', 'Age_Adult', 'Age_Senior', 'Fare_Low', 'Fare_Medium', 'Fare_High', 'Fare_Very_High']


Prepare Features and Target

Why Separate X and y?
Machine learning models expect:
- **X** (features): The input variables used to make predictions
- **y** (target): The outcome we're trying to predict (Survived)

In [5]:
# Separate features and target
X = train_df.drop('Survived', axis=1)
y = train_df['Survived']

# Test set (no target - that's what we predict)
X_test_final = test_df.copy()

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Target distribution:\n{y.value_counts(normalize=True)}")

Features shape: (891, 34)
Target shape: (891,)
Target distribution:
Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64


Train-Validation Split

Why Split the Training Data?
We need to evaluate our model on data it hasn't seen during training. This gives us 
an honest estimate of how well it will perform on the actual test set.

- **Training set (80%)**: Used to train the model
- **Validation set (20%)**: Used to evaluate and compare models

Stratified Splitting
We use stratified splitting to ensure the same proportion of survivors in both 
train and validation sets. This is important because our classes are imbalanced (~38% survived).

In [7]:
# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=RANDOM_STATE,
    stratify=y  # Maintain class proportions
)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"\nTraining target distribution:\n{y_train.value_counts(normalize=True)}")
print(f"\nValidation target distribution:\n{y_val.value_counts(normalize=True)}")

Training set: (712, 34)
Validation set: (179, 34)

Training target distribution:
Survived
0    0.616573
1    0.383427
Name: proportion, dtype: float64

Validation target distribution:
Survived
0    0.614525
1    0.385475
Name: proportion, dtype: float64


Feature Scaling

Why Scale Features?
Some algorithms (like Logistic Regression) are sensitive to feature scales:
- Age ranges from 0-80
- Fare ranges from 0-512
- Binary features are just 0 or 1

**StandardScaler** transforms features to have mean=0 and std=1.

Important: Fit on Training Only!
We fit the scaler on training data and transform both train and validation. 
This prevents "data leakage" - using information from the validation set during training.

In [8]:
# Initialize scaler
scaler = StandardScaler()

# Fit on training data, transform both
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test_final)

# Convert back to DataFrames for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns, index=X_val.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_final.columns)

print("Feature scaling complete!")
print(f"\nScaled training data sample (first 3 rows, first 5 columns):")
print(X_train_scaled.iloc[:3, :5])

Feature scaling complete!

Scaled training data sample (first 3 rows, first 5 columns):
       Pclass       Age     SibSp     Parch      Fare
692  0.829568 -0.322182 -0.465084 -0.466183  0.513812
481 -0.370945  0.053575 -0.465084 -0.466183 -0.662563
527 -1.571457  0.805089 -0.465084 -0.466183  3.955399


Cross-Validation Setup

Why Cross-Validation?
A single train-validation split can give unstable results depending on which samples 
end up in which set. Cross-validation gives us a more reliable estimate by:

1. Splitting data into K folds
2. Training K times, each time using a different fold as validation
3. Averaging the results

We'll use **5-fold stratified cross-validation** for model comparison.

In [9]:
# Set up cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Helper function to evaluate models
def evaluate_model(model, X_train, y_train, X_val, y_val, model_name):
    """
    Train model and return comprehensive evaluation metrics.
    """
    # Train
    model.fit(X_train, y_train)
    
    # Predict
    y_pred = model.predict(X_val)
    y_pred_proba = model.predict_proba(X_val)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Calculate metrics
    metrics = {
        'Model': model_name,
        'Accuracy': accuracy_score(y_val, y_pred),
        'Precision': precision_score(y_val, y_pred),
        'Recall': recall_score(y_val, y_pred),
        'F1': f1_score(y_val, y_pred),
        'ROC_AUC': roc_auc_score(y_val, y_pred_proba) if y_pred_proba is not None else None
    }
    
    # Cross-validation score (on full training set)
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    metrics['CV_Mean'] = cv_scores.mean()
    metrics['CV_Std'] = cv_scores.std()
    
    return metrics, y_pred, y_pred_proba

Model 1: Logistic Regression (Baseline)

Why Start with Logistic Regression?
Logistic Regression is the classic baseline for binary classification:
- **Simple and interpretable**: Coefficients show feature importance
- **Fast to train**: Works well even on larger datasets
- **Sets a benchmark**: If complex models don't beat this, they're not worth the complexity

How It Works
Logistic regression models the probability of survival as:
P(Survived=1) = 1 / (1 + e^-(β₀ + β₁x₁ + ... + βₙxₙ))

Each coefficient (β) tells us how that feature affects survival odds.

In [10]:
# Train Logistic Regression
lr_model = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)

lr_metrics, lr_pred, lr_proba = evaluate_model(
    lr_model, X_train_scaled, y_train, X_val_scaled, y_val, 'Logistic Regression'
)

print("Logistic Regression Results:")
print(f"  Accuracy:  {lr_metrics['Accuracy']:.4f}")
print(f"  Precision: {lr_metrics['Precision']:.4f}")
print(f"  Recall:    {lr_metrics['Recall']:.4f}")
print(f"  F1 Score:  {lr_metrics['F1']:.4f}")
print(f"  ROC AUC:   {lr_metrics['ROC_AUC']:.4f}")
print(f"  CV Score:  {lr_metrics['CV_Mean']:.4f} (+/- {lr_metrics['CV_Std']:.4f})")

Logistic Regression Results:
  Accuracy:  0.8492
  Precision: 0.8182
  Recall:    0.7826
  F1 Score:  0.8000
  ROC AUC:   0.8684
  CV Score:  0.8132 (+/- 0.0329)


In [11]:
# Examine logistic regression coefficients
lr_coef = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': lr_model.coef_[0]
}).sort_values('Coefficient', key=abs, ascending=False)

print("Top 10 Most Important Features (by coefficient magnitude):")
print(lr_coef.head(10).to_string(index=False))

Top 10 Most Important Features (by coefficient magnitude):
     Feature  Coefficient
 Sex_encoded     0.816813
      Pclass    -0.490991
    Title_Mr    -0.459007
       SibSp    -0.434395
  FamilySize    -0.369636
Title_Master     0.368865
   Title_Mrs     0.356277
         Age    -0.252910
      Deck_E     0.236296
Deck_Unknown    -0.232623


Model 2: Random Forest

Why Random Forest?
Random Forest is an ensemble method that addresses decision tree weaknesses:
- **Reduces overfitting**: Averages many trees to reduce variance
- **Handles non-linear relationships**: Trees can capture complex patterns
- **No scaling required**: Tree-based models don't need feature scaling
- **Feature importance**: Built-in importance scores

How It Works
1. Create many decision trees, each trained on a random subset of data and features
2. Each tree "votes" on the prediction
3. Final prediction = majority vote

In [12]:
# Train Random Forest (using unscaled data - trees don't need scaling)
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

rf_metrics, rf_pred, rf_proba = evaluate_model(
    rf_model, X_train, y_train, X_val, y_val, 'Random Forest'
)

print("Random Forest Results:")
print(f"  Accuracy:  {rf_metrics['Accuracy']:.4f}")
print(f"  Precision: {rf_metrics['Precision']:.4f}")
print(f"  Recall:    {rf_metrics['Recall']:.4f}")
print(f"  F1 Score:  {rf_metrics['F1']:.4f}")
print(f"  ROC AUC:   {rf_metrics['ROC_AUC']:.4f}")
print(f"  CV Score:  {rf_metrics['CV_Mean']:.4f} (+/- {rf_metrics['CV_Std']:.4f})")

Random Forest Results:
  Accuracy:  0.8101
  Precision: 0.7966
  Recall:    0.6812
  F1 Score:  0.7344
  ROC AUC:   0.8437
  CV Score:  0.8217 (+/- 0.0320)


In [13]:
# Random Forest feature importance
rf_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("Top 10 Most Important Features (Random Forest):")
print(rf_importance.head(10).to_string(index=False))

Top 10 Most Important Features (Random Forest):
     Feature  Importance
    Title_Mr    0.169468
 Sex_encoded    0.141159
        Fare    0.102660
         Age    0.084280
      Pclass    0.077119
Deck_Unknown    0.052334
   Title_Mrs    0.050421
  Title_Miss    0.047166
  FamilySize    0.046621
       SibSp    0.031831


Model 3: XGBoost

Why XGBoost?
XGBoost (eXtreme Gradient Boosting) is often the winning algorithm in competitions:
- **Sequential learning**: Each tree corrects errors from previous trees
- **Regularization**: Built-in L1/L2 regularization prevents overfitting
- **Handles missing values**: Can learn optimal handling of missing data
- **Fast**: Optimized implementation

How It Works
1. Start with a simple prediction (e.g., overall survival rate)
2. Build a tree to predict the errors (residuals)
3. Add the tree's predictions to improve the model
4. Repeat, focusing on samples that are still being predicted incorrectly

In [14]:
# Train XGBoost
xgb_model = XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=RANDOM_STATE,
    eval_metric='logloss',
    use_label_encoder=False
)

xgb_metrics, xgb_pred, xgb_proba = evaluate_model(
    xgb_model, X_train, y_train, X_val, y_val, 'XGBoost'
)

print("XGBoost Results:")
print(f"  Accuracy:  {xgb_metrics['Accuracy']:.4f}")
print(f"  Precision: {xgb_metrics['Precision']:.4f}")
print(f"  Recall:    {xgb_metrics['Recall']:.4f}")
print(f"  F1 Score:  {xgb_metrics['F1']:.4f}")
print(f"  ROC AUC:   {xgb_metrics['ROC_AUC']:.4f}")
print(f"  CV Score:  {xgb_metrics['CV_Mean']:.4f} (+/- {xgb_metrics['CV_Std']:.4f})")

XGBoost Results:
  Accuracy:  0.8101
  Precision: 0.7692
  Recall:    0.7246
  F1 Score:  0.7463
  ROC AUC:   0.8355
  CV Score:  0.8329 (+/- 0.0198)


In [15]:
# XGBoost feature importance
xgb_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("Top 10 Most Important Features (XGBoost):")
print(xgb_importance.head(10).to_string(index=False))

Top 10 Most Important Features (XGBoost):
       Feature  Importance
   Sex_encoded    0.235900
      Title_Mr    0.143569
        Pclass    0.059179
  Deck_Unknown    0.055967
    Title_Rare    0.043332
    FamilySize    0.034156
        Deck_D    0.033052
Fare_Very_High    0.028373
        Deck_E    0.027556
      Fare_Low    0.022939


Model Comparison

Now let's compare all three models side-by-side to determine which performs best.


In [16]:
# Create comparison DataFrame
results = pd.DataFrame([lr_metrics, rf_metrics, xgb_metrics])
results = results.set_index('Model')

print("Model Comparison:")
print(results.round(4).to_string())

Model Comparison:
                     Accuracy  Precision  Recall      F1  ROC_AUC  CV_Mean  CV_Std
Model                                                                             
Logistic Regression    0.8492     0.8182  0.7826  0.8000   0.8684   0.8132  0.0329
Random Forest          0.8101     0.7966  0.6812  0.7344   0.8437   0.8217  0.0320
XGBoost                0.8101     0.7692  0.7246  0.7463   0.8355   0.8329  0.0198


In [24]:
# Visualize model comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Main metrics comparison
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1']
x = np.arange(len(metrics_to_plot))
width = 0.25

bars1 = axes[0].bar(x - width, results.loc['Logistic Regression', metrics_to_plot], width, label='Logistic Regression')
bars2 = axes[0].bar(x, results.loc['Random Forest', metrics_to_plot], width, label='Random Forest')
bars3 = axes[0].bar(x + width, results.loc['XGBoost', metrics_to_plot], width, label='XGBoost')

axes[0].set_ylabel('Score')
axes[0].set_title('Model Performance Comparison')
axes[0].set_xticks(x)
axes[0].set_xticklabels(metrics_to_plot)
axes[0].legend()
axes[0].set_ylim(0.6, 1.0)

# Plot 2: Cross-validation scores with error bars
models = ['Logistic Regression', 'Random Forest', 'XGBoost']
cv_means = [results.loc[m, 'CV_Mean'] for m in models]
cv_stds = [results.loc[m, 'CV_Std'] for m in models]

axes[1].bar(models, cv_means, yerr=cv_stds, capsize=5, color=['steelblue', 'forestgreen', 'coral'])
axes[1].set_ylabel('CV Accuracy')
axes[1].set_title('Cross-Validation Scores (5-Fold)')
axes[1].set_ylim(0.7, 0.9)

# Add value labels
for i, (mean, std) in enumerate(zip(cv_means, cv_stds)):
    axes[1].text(i, mean + std + 0.01, f'{mean:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.savefig('../reports/figures/model_comparison.png')
plt.close()

Confusion Matrices

Understanding the Confusion Matrix
A confusion matrix shows us where our model makes mistakes:
- **True Positives (TP)**: Correctly predicted survivors
- **True Negatives (TN)**: Correctly predicted non-survivors
- **False Positives (FP)**: Predicted survival but actually died (Type I error)
- **False Negatives (FN)**: Predicted death but actually survived (Type II error)

In [25]:
# Plot confusion matrices for all models
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

predictions = [
    ('Logistic Regression', lr_pred),
    ('Random Forest', rf_pred),
    ('XGBoost', xgb_pred)
]

for ax, (name, y_pred) in zip(axes, predictions):
    cm = confusion_matrix(y_val, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['Died', 'Survived'],
                yticklabels=['Died', 'Survived'])
    ax.set_title(f'{name}')
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')

plt.tight_layout()
plt.savefig('../reports/figures/confusion_matrices.png')
plt.close()

ROC Curves

Why ROC Curves Matter
The ROC (Receiver Operating Characteristic) curve shows the trade-off between:
- **True Positive Rate** (Sensitivity): How many survivors did we correctly identify?
- **False Positive Rate** (1 - Specificity): How many non-survivors did we incorrectly flag as survivors?

**AUC (Area Under Curve)**: A single number summarizing performance. 
 1.0 = perfect, 0.5 = random guessing.

In [26]:
# Plot ROC curves
fig, ax = plt.subplots(figsize=(8, 6))

probabilities = [
    ('Logistic Regression', lr_proba, lr_metrics['ROC_AUC']),
    ('Random Forest', rf_proba, rf_metrics['ROC_AUC']),
    ('XGBoost', xgb_proba, xgb_metrics['ROC_AUC'])
]

for name, proba, auc in probabilities:
    fpr, tpr, _ = roc_curve(y_val, proba)
    ax.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})')

ax.plot([0, 1], [0, 1], 'k--', label='Random Guess')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curves - Model Comparison')
ax.legend(loc='lower right')

plt.tight_layout()
plt.savefig('../reports/figures/rocauc_scores.png')
plt.close()

Feature Importance Comparison

Let's see which features are most important across our models. 
This helps us understand what drives survival predictions.

In [27]:
# Combine feature importance from all models
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'LR_Importance': np.abs(lr_model.coef_[0]),  # Use absolute value for comparison
    'RF_Importance': rf_model.feature_importances_,
    'XGB_Importance': xgb_model.feature_importances_
})

# Normalize to 0-1 scale for comparison
for col in ['LR_Importance', 'RF_Importance', 'XGB_Importance']:
    importance_df[col] = importance_df[col] / importance_df[col].max()

# Calculate average importance
importance_df['Avg_Importance'] = importance_df[['LR_Importance', 'RF_Importance', 'XGB_Importance']].mean(axis=1)
importance_df = importance_df.sort_values('Avg_Importance', ascending=False)

# Plot top 15 features
top_features = importance_df.head(15)

fig, ax = plt.subplots(figsize=(12, 8))

x = np.arange(len(top_features))
width = 0.25

ax.barh(x - width, top_features['LR_Importance'], width, label='Logistic Regression')
ax.barh(x, top_features['RF_Importance'], width, label='Random Forest')
ax.barh(x + width, top_features['XGB_Importance'], width, label='XGBoost')

ax.set_yticks(x)
ax.set_yticklabels(top_features['Feature'])
ax.set_xlabel('Normalized Importance')
ax.set_title('Top 15 Features by Importance (All Models)')
ax.legend()
ax.invert_yaxis()  # Most important at top

plt.tight_layout()
plt.savefig('../reports/figures/feature_importance.png')
plt.close()

Select Best Model

Based on our evaluation, let's select the best performing model for final predictions.


In [28]:
# Determine best model based on CV score (most reliable metric)
best_model_name = results['CV_Mean'].idxmax()
print(f"Best model based on CV accuracy: {best_model_name}")
print(f"CV Score: {results.loc[best_model_name, 'CV_Mean']:.4f} (+/- {results.loc[best_model_name, 'CV_Std']:.4f})")

# Select the best model object
models = {
    'Logistic Regression': lr_model,
    'Random Forest': rf_model,
    'XGBoost': xgb_model
}
best_model = models[best_model_name]

# Also determine if we need scaled data
use_scaled = best_model_name == 'Logistic Regression'

Best model based on CV accuracy: XGBoost
CV Score: 0.8329 (+/- 0.0198)


Retrain on Full Training Data

Now that we've selected our best model, we retrain it on the **full** training dataset 
(not just the 80% training split) to maximize the information it learns from.

In [29]:
# Retrain best model on full training data
if use_scaled:
    # Scale full training data
    scaler_full = StandardScaler()
    X_full_scaled = scaler_full.fit_transform(X)
    best_model.fit(X_full_scaled, y)
    print(f"Retrained {best_model_name} on full scaled training data ({len(X)} samples)")
else:
    best_model.fit(X, y)
    print(f"Retrained {best_model_name} on full training data ({len(X)} samples)")

Retrained XGBoost on full training data (891 samples)


Generate Test Predictions

Now we generate predictions for the test set - these are the passengers whose survival 
we're trying to predict for the Kaggle competition.

In [30]:
# Generate predictions for test set
if use_scaled:
    X_test_for_pred = scaler_full.transform(X_test_final)
else:
    X_test_for_pred = X_test_final

test_predictions = best_model.predict(X_test_for_pred)

print(f"Generated {len(test_predictions)} predictions")
print(f"Predicted survival rate: {test_predictions.mean():.2%}")
print(f"(Training survival rate was: {y.mean():.2%})")

Generated 418 predictions
Predicted survival rate: 34.69%
(Training survival rate was: 38.38%)
